# Job Interview Wihth OpenAI's GPT model


## What you need to get started

This notebook needs a `.env` file in the same folder with the following data:
```
SPEECH_KEY=<Cognitive services key>
SPEECH_REGION=<cognitive service region>
SPEECH_LANGUAGE=<cognitive services language>
OPENAI_KEY=<openAI API key>
```

## How do I get the speech key and region?

You need a valid [Azure](https://portal.azure.com/) subscription in order to get those keys.

Once you do, follow these steps:
- (optional) Create a new Azure Resource Group and call it `speech`
- From the resource group, click on **Create** and search for `Cognitive  Services`
- Select the first one anc click **Create**
- Choose your subscription and resource group. Also provide a name and choose the pricing tier. You can find more information about Cognitive services' pricing [here](https://azure.microsoft.com/en-us/pricing/details/cognitive-services/speech-services/)
- Leave everything else as default and click **Review + Create** then click **Create** if there are no issues.
- Once the service is created, either click on **Go to Resource** or navigate to your resource
- Click on **Keys and Endpoint** on the left pane
- From there you can copy your key and region, and paste it in the `.env` file mentioned above.

## How do I get the OpenAI Key?

You need a valid paid subscription to OpenAI. More information about costs [here](https://openai.com/api/pricing/)

- Go to [OpenAI's API website](https://openai.com/api/)
- **Log In** or **Sign Up** if you haven't done so already
- Click on your profile in the right hand side of the screen
- Click on **View API Keys**
- Click on **Create New Secret Key**
- Copy and paste the value in the `.env` file mentioned above.

## How to use this notebook?

In the code box below, replace the `jobTitle` with the job title you want to practice with.
Enter the `numberOfQuestions` you want to practice on.

Run all the cells.

When the AI asks you a question, there will be a shot tone to indicate that it's ready to listen to you. If you stop talking for 2.5s, it will automatically assume you are done answering and move to the next question.

# Define the interview parameters


In [37]:
jobTitle = "Random" # If you enter 'Random' as the job title, it will choose one randomly from the input file 'Job Titles.txt'
numberOfQuestions = 5 # The number of questions will be:
                      #   - 1 Intro question of the style: tell me about yourself
                      #   - <numberOfQuestions> randomly generated questions
                      #   - <numberOfQuestions> followup questions to the previously generated questions
                      # The total number of questions asked will be: <numberOfQuestions> x 2 + 1
                      # For example, for numberOfQuestions = 5, the total number of questions asked will be 11


In [38]:
import random

if jobTitle == 'Random':
    # Open the file in read mode
    with open('Job Titles.txt', 'r') as f:
        # Read all lines of the file into a list
        jobs = f.readlines()

    # Strip the '\n' character from each element in the list
    jobs = [job.strip() for job in jobs]

    # Get the unique jobs from the input file
    jobs = list(set(jobs))

    # Get a random job title from the list.
    jobIndex = random.randint(0,len(jobs)-1)
    jobTitle = jobs[jobIndex]

# Install libraries


In [39]:
! pip3 install azure-cognitiveservices-speech
! pip3 install python-dotenv
! pip3 install openai
! pip3 install simpleaudio


# Initialize


In [40]:
from dotenv import load_dotenv
import re
import os
import openai
from datetime import date, datetime, timedelta


load_dotenv(override=True)

settings = {
    'speechKey': os.environ.get('SPEECH_KEY'),
    'region': os.environ.get('SPEECH_REGION'),
    # Feel free to hardcode the language
    'language': os.environ.get('SPEECH_LANGUAGE'),
    'openAIKey': os.environ.get('OPENAI_KEY')
}
# Load your API key from an environment variable or secret management service
openai.api_key = settings['openAIKey']

# Some sounds need to be generated over and over, like "thank you" or "I didn't get that".
# There is no need to waste money re-generating them, so we will keep track of them here
already_spoken = {}

output_folder = f'./Output/{jobTitle}-{datetime.now().strftime("%Y%m%d_%H%M%S")}/'

os.makedirs(output_folder)


# Utility functions


In [41]:
def complete_openai(prompt, token=50,stop=""):
    """
    Generates completions for a given prompt using the OpenAI API.
    
    Parameters:
    - prompt: The prompt for which completions are generated.
    - token: The maximum number of tokens (words) in the generated completions.
    - stop: A string that, if encountered in the generated completions, will cause the function to stop generating more completions.
    
    Returns:
    A list of strings containing the generated completions.
    """
    try:
        if (not stop):
            response = openai.Completion.create(
                model="text-davinci-003",
                prompt=prompt,
                temperature=0.9,
                max_tokens=token,
                top_p=1,
                presence_penalty = 1.5,
                frequency_penalty = 1.5
            )
            lines = response.to_dict_recursive()['choices'][0]['text'].split("\n")
            response = list(filter(lambda x: x != '', lines))
            return response
        else:
            response = openai.Completion.create(
                model="text-davinci-003",
                prompt=prompt,
                temperature=0.9,
                max_tokens=token,
                top_p=1,
                presence_penalty = 1.5,
                frequency_penalty = 1.5,
                stop=[stop]
            )
            lines = response.to_dict_recursive()['choices'][0]['text'].split("\n")
            response = list(filter(lambda x: x != '', lines))
            return response
    except Exception as e:
        print("An exception of type", type(e), "occurred with the message:", e)



In [42]:
# Play a sound to indicate recording session is on or about to be stopped.
import simpleaudio as sa
import numpy as np

def play_sound():
    # set the frequency and duration
    frequency = 440
    duration = 0.1  # in seconds

    # create a waveform
    sample_rate = 44100
    amplitude = 16000

    waveform = np.sin(2 * np.pi * np.arange(sample_rate * duration) * frequency / sample_rate)
    waveform = (waveform * amplitude).astype(np.int16)

    # create a simpleaudio object
    audio = sa.play_buffer(waveform, 1, 2, sample_rate)

    # wait for the waveform to finish playing
    audio.wait_done()

In [43]:
import azure.cognitiveservices.speech as speechsdk
import time

prop = False


def Start_recording_answer():

    # Creates an instance of a speech config with specified subscription key and service region.
    speech_config = speechsdk.SpeechConfig(
        subscription=settings['speechKey'], region=settings['region'])

    speech_config.request_word_level_timestamps()
    speech_config.set_property(
        property_id=speechsdk.PropertyId.SpeechServiceResponse_OutputFormatOption, value="detailed")

    # Creates a speech recognizer using the default microphone (built-in).
    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)

    speech_recognizer = speechsdk.SpeechRecognizer(
        speech_config=speech_config, audio_config=audio_config)

    # initialize some variables
    results = []
    done = False
    
    # update the last time speech was detected. 
    def speech_detected():
        nonlocal lastSpoken
        lastSpoken = int(datetime.now().timestamp() * 1000)

    # Event handler to add event to the result list
    def handleResult(evt):
        import json
        nonlocal results
        nonlocal lastSpoken
        results.append(json.loads(evt.result.json))

        # print the result (optional, otherwise it can run for a few minutes without output)
        # print('RECOGNIZED: {}'.format(evt))
        speech_detected()

        # result object
        res = {'text': evt.result.test, 'timestamp': evt.result.offset,
               'duration': evt.result.duration, 'raw': evt.result}

        if (evt.result.text != ""):
            results.append(res)
            
            # print(evt.result)

    # Event handler to check if the recognizer is done

    def stop_cb(evt):
        # print('CLOSING on {}'.format(evt))
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True

    # Connect callbacks to the events fired by the speech recognizer & displays the info/status
    # Ref:https://docs.microsoft.com/en-us/python/api/azure-cognitiveservices-speech/azure.cognitiveservices.speech.eventsignal?view=azure-python
    speech_recognizer.recognizing.connect(lambda evt: speech_detected())
    # speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
    speech_recognizer.session_started.connect(
        lambda evt: print('SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(
        lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(
        lambda evt: print('CANCELED {}'.format(evt)))
    speech_recognizer.recognized.connect(handleResult)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)
    
    # Start speech recognition
    result_future = speech_recognizer.start_continuous_recognition_async()
    result_future.get()

    # Play sound to indicate that the recording session is on.
    play_sound()

    lastSpoken = int(datetime.now().timestamp() * 1000)

    # Wait for speech recognition to complete
    while not done:
        time.sleep(1)
        now = int(datetime.now().timestamp() * 1000)
        inactivity = now - lastSpoken
        # print(inactivity)
        if (inactivity > 1000): # After 1 second of no speech detected, play a sound to indicate the recoding session could close.
            play_sound()
        if (inactivity > 3000): # Close the recoding session if no input is detected after 3s
            print('Stopping async recognition.')
            speech_recognizer.stop_continuous_recognition_async()
            speak("Thank you!")
            while not done:
                time.sleep(1)

    output = ""
    for res in results:
        output += res['NBest'][0]['Display']

    return results


In [65]:
import azure.cognitiveservices.speech as speechsdk



def speak(text,silent=False):

    if text in already_spoken: # if the speech was already synthetized
        if not silent:
            play_obj = sa.WaveObject.from_wave_file(already_spoken[text]).play()
            play_obj.wait_done()
        return

    # This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
    speech_config = speechsdk.SpeechConfig(
        subscription=settings['speechKey'], region=settings['region'])
    # audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
    file_name = f'{output_folder}/{datetime.now().strftime("%Y%m%d_%H%M%S")}.wav'
    audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True,filename=file_name)

    # The language of the voice that speaks.
    speech_config.speech_synthesis_voice_name = 'en-US-JennyNeural'

    speech_synthesizer = speechsdk.SpeechSynthesizer(
        speech_config=speech_config, audio_config=audio_config)

    speech_synthesis_result = speech_synthesizer.speak_text(text) #.get()

    if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print("Speech synthesized for text [{}]".format(text))
        if not silent:
            play_obj = sa.WaveObject.from_wave_file(file_name).play()
            play_obj.wait_done()
        already_spoken[text]=file_name
    elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_synthesis_result.cancellation_details
        print("Speech synthesis canceled: {}".format(
            cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            if cancellation_details.error_details:
                print("Error details: {}".format(
                    cancellation_details.error_details))
                print("Did you set the speech resource key and region values?")

def speak_ssml(text):
    # This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
    speech_config = speechsdk.SpeechConfig(
        subscription=settings['speechKey'], region=settings['region'])
    # audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)

    # The language of the voice that speaks.
    speech_config.speech_synthesis_voice_name = 'en-US-JennyNeural'

    speech_synthesizer = speechsdk.SpeechSynthesizer(
        speech_config=speech_config, audio_config=None)

    speech_synthesis_result = speech_synthesizer.speak_ssml(text) #.speak_text(text) #.get()

    if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print("Speech synthesized for text [{}]".format(text))
    elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_synthesis_result.cancellation_details
        print("Speech synthesis canceled: {}".format(
            cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            if cancellation_details.error_details:
                print("Error details: {}".format(
                    cancellation_details.error_details))
                print("Did you set the speech resource key and region values?")


In [45]:
def get_feedback(interviewer, interviewee):
    prompt = """
write a paragraph to the interviewee describing what they did well in their response, and how they can improve:

interviewer: {0}

interviewee: {1}
""".format(interviewer, interviewee)
    # print(prompt)
    res = complete_openai(prompt=prompt, token=int(4000-len(prompt.split())*1.75))
    res = ("\n".join(res))
    return {'feedback':res,'score':get_score(interviewer=interviewer,interviewee=interviewee)}

def get_score(interviewer, interviewee):
    prompt = """
on a scale from 1 to 10 where 1 is the worst performance and 10 is the best, rater the interviewee's answer (only give me a number, no words):

interviewer: {0}

interviewee: {1}
""".format(interviewer, interviewee)
    # print(prompt)
    res = complete_openai(prompt=prompt, token=int(4000-len(prompt.split())*1.75))
    res = ("\n".join(res))
    return res

In [46]:
def openai_filter(item):
    if item == '.':
        return False
    return True

# Interview


In [47]:
dialog = []
askedQuestions = ""
questions = complete_openai(prompt=f'Give me a list of {numberOfQuestions} questions an interviewer may ask for a {jobTitle} job interview. Provide them each on a new line with no dashes or numbers at the beginning',token=3000)
questions = list(filter(openai_filter,questions))
# wait = input()

In [48]:
intro = complete_openai(
    prompt="Write the very first question for an interviewer interviewing an interviewee applying for a {0} job. Make sure to say a greeting, and also describe the role briefly.".format(jobTitle), token=3000)
introDialog = ("\n".join(intro)).replace("Interviewer: ", "")
speak(introDialog)
askedQuestions+="{0}\n".format(introDialog)
answer = Start_recording_answer()
dialog += [{
    'interviewer': introDialog,
    'interviewee': answer[0]['DisplayText'],
    'feedback': '',
    'score': ''
}]

Speech synthesized for text [Good morning! Thank you for taking the time to speak with me today. I am very interested in learning more about your experience and qualifications as it relates to this Cloud Data Analyst role. To begin, can you tell me a bit about what motivated you to apply for this job?]
SESSION STARTED: SessionEventArgs(session_id=30299e8580fa430a9152e80966a94262)
Stopping async recognition.
SESSION STOPPED SessionEventArgs(session_id=30299e8580fa430a9152e80966a94262)
Speech synthesized for text [Thank you!]


In [49]:
askedQuestions = ""
askedQuestions+= "{0}\n".format(introDialog)
askedQuestions

'Good morning! Thank you for taking the time to speak with me today. I am very interested in learning more about your experience and qualifications as it relates to this Cloud Data Analyst role. To begin, can you tell me a bit about what motivated you to apply for this job?\n'

In [50]:
for i in range(0, numberOfQuestions*2):
    prompt = """
Provide the question the interviewer will ask to more effectively assess the candidate's experience and knowledge as it relates to a {2} role based on the following information (please only return the question):
context: this is a job interview for a {2} role.
Previously asked questions: {3}
previous conversation:
interviewer: {0}

interviewee: {1}
|
""".format(dialog[len(dialog)-1]['interviewer'], dialog[len(dialog)-1]['interviewee'], jobTitle, askedQuestions, stop="|")
    interviewer = complete_openai(
        prompt=prompt,
        token=600
    )
    questions.append(interviewer[0])
    # askedQuestions += "{0}\n".format(interviewer[0])
    speak(interviewer[0])
    answer = ""
    i = 0
    while (len(answer) < 20):
        if (i > 0):
            speak("Sorry, I did not get that.")
        answer = (Start_recording_answer())[0]['DisplayText']
        i += 1
    dialog += [{
        'interviewer': interviewer[0],
        'interviewee': answer,
        'feedback': "",
        'score': ""
    }]


Speech synthesized for text [1. What experience do you have in developing and implementing cloud data solutions? ]
SESSION STARTED: SessionEventArgs(session_id=364d8bc6fb6e4509b426314ab336d131)
Stopping async recognition.
SESSION STOPPED SessionEventArgs(session_id=364d8bc6fb6e4509b426314ab336d131)
Speech synthesized for text [2. How experienced are you with generating insights from large datasets?  ]
SESSION STARTED: SessionEventArgs(session_id=e70429e9a2fc486e8a7281e73c96af7f)
Stopping async recognition.
SESSION STOPPED SessionEventArgs(session_id=e70429e9a2fc486e8a7281e73c96af7f)
Speech synthesized for text [3. Are there any specific technologies or platforms that you feel comfortable working with when it comes to cloud data analysis? ]
SESSION STARTED: SessionEventArgs(session_id=794862d044ae44a681ed8f02040fada7)
Stopping async recognition.
SESSION STOPPED SessionEventArgs(session_id=794862d044ae44a681ed8f02040fada7)
Speech synthesized for text [4. Describe a time where your analys

In [67]:
speak("Here is your feedback:",silent=True)
for interaction in dialog:
    feedback = get_feedback(
        interaction['interviewer'], interaction['interviewee'])
    interaction['feedback'] = feedback['feedback']
    interaction['score'] = feedback['score']
    speak("""
        For the question: {0}
        The feedback is: {1}
        
        Overall, I would rate your performance for this question a {2} out of 10.  
""".format(interaction['interviewer'], interaction['feedback'],interaction['score']))
    # break


Speech synthesized for text [
        For the question: Good morning! Thank you for taking the time to speak with me today. I am very interested in learning more about your experience and qualifications as it relates to this Cloud Data Analyst role. To begin, can you tell me a bit about what motivated you to apply for this job?
        The feedback is: You did a great job of expressing your motivation for applying to this role. You clearly demonstrated that you have experience working in the data analyst space and with cloud data, which will likely be beneficial when it comes time to apply your knowledge on the job. Perhaps next time, you can use concrete examples to further demonstrate how these experiences could come into play as it relates specifically to this Cloud Data Analyst position.
        
        Overall, I would rate your performance for this question a 8 out of 10.  
]
Speech synthesized for text [
        For the question: 1. What experience do you have in developing and

In [66]:

output = ""
for interaction in dialog:
    output+="""
Question:
{0}

Feedback:
{1}

Score:
{2}/10
-------------------------------------
""".format(interaction['interviewer'],interaction['feedback'],interaction['score'])

# Open a file in write mode
with open(f'{output_folder}/transcript-feedback.txt', "w") as f:
    # Write the string to the file
    f.write(output)

In [61]:
for i in range(1,len(dialog)):
    print(dialog[i]['interviewer'])
    print(questions[i-1])
    # dialog[i]['interviewer'] = questions[i-1]
    print('---')

1. What experience do you have in developing and implementing cloud data solutions? 
1. What experience do you have in developing and implementing cloud data solutions? 
---
2. How experienced are you with generating insights from large datasets?  
2. How experienced are you with generating insights from large datasets?  
---
3. Are there any specific technologies or platforms that you feel comfortable working with when it comes to cloud data analysis? 
3. Are there any specific technologies or platforms that you feel comfortable working with when it comes to cloud data analysis? 
---
4. Describe a time where your analysis was used to make a successful business decision for an organization. 
4. Describe a time where your analysis was used to make a successful business decision for an organization. 
---
5. What does best practice look like when analyzing, interpreting, and presenting complex datasets in the Cloud environment?
5. What does best practice look like when analyzing, interpre

In [53]:
speak("Here is your feedback:")
# speak_ssml("""
# <speak
#     version="1.0"
#     xmlns="https://www.w3.org/2001/10/synthesis"
#     xmlns:mstts="https://www.w3.org/2001/mstts"
#     xml:lang="en-US"
#     >
#   <voice name="en-US-JennyNeural">
#     <express-as style="calm">
#         For the question:
#     </express-as>
#     <mstts:express-as role="YoungAdultMale" style="calm">
#         Good morning! Thank you for taking the time to meet with us today. We are currently looking to fill a role as Cloud Engineer, which will involve designing and building our company’s cloud-based infrastructure. To start off this interview, can you tell me about your experience working on cloud systems?
#     </mstts:express-as>
#     <mstts:express-as role="YoungAdultFemale" style="calm">
#         The feedback is:
#     </mstts:express-as>
#     <mstts:express-as role="YoungAdultMale" style="calm">
#         Your response was strong and detailed; you gave us clear examples of your experience working on cloud systems. You mentioned that you have been doing this for two years, which is important context to show the scope of knowledge and expertise. In the future, it would be great to provide an example or two about a project in which you utilized these skills so we can better understand how involved and successful you are at utilizing the cloud.
#     </mstts:express-as>
#     <mstts:express-as role="YoungAdultFemale" style="calm">
#         Overall, I would rate your performance for this question a 9 out of 10.
#     </mstts:express-as>
#   </voice>
# </speak> 
# """)